In [2]:
from bs4 import BeautifulSoup
import requests
import webbrowser
from urllib.parse import urljoin, unquote
import json

# Step 1: Send an HTTP request to the webpage
url = 'https://archive.org/download/hogwarts-legacy-voice-files'  # Replace with the URL you want to scrape
response = requests.get(url)

# Step 2: Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')


In [3]:
# Extract string between /download/ and .rar/
def extract_string_between(text, start, end):
    start_index = text.find(start) + len(start)
    end_index = text.find(end, start_index)
    return text[start_index:end_index]

# Step 3: Find all <a> tags containing the text "View content"
folders = []
for a_tag in soup.find_all('a', string="View Contents"):
    href = a_tag.get('href')
    if href:
        # Convert relative URLs to absolute URLs
        full_url = urljoin(url, href)
        extracted_string = extract_string_between(full_url, '/download/', '.rar/')
        folders.append(extracted_string)


In [4]:
url2 = 'https://ia601601.us.archive.org/view_archive.php?archive=/29/items/hogwarts-legacy-voice-files/'

In [5]:
def parseUrl(url):
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

# Save links json file
# Create a dictionary
data = {}
for folder in folders:
    data[unquote(folder)] = []
    final_link = url2 + folder + ".rar"
    soup = parseUrl(final_link)
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href')
        if href:
            # Convert relative URLs to absolute URLs
            urls = urljoin(url, href)
            if urls.endswith(".wav"):
                data[unquote(folder)].append(urls)  

# Save the dictionary to a JSON file
with open('links.json', 'w') as file:
    json.dump(data, file, indent=4)

In [ ]:
urls = [
    "https://downloads.khinsider.com/game-soundtracks/album/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023",
    "https://downloads.khinsider.com/game-soundtracks/album/hogwarts-legacy-original-video-game-soundtrack-2023"
]
data = {}
for url in urls:
    soup = parseUrl(url)
    title = soup.find("h2").text
    files = []
    for a_tag in soup.find_all('a'):
        href = a_tag.get('href')
        if href:
            # Convert relative URLs to absolute URLs
            full_url = urljoin(url, href)
            if full_url.endswith('.mp3'):
                files.append(full_url)
    files = list(set(files))
    srcs = []
    for file in files:
        soup = parseUrl(file)
        for tag in soup.find_all("audio"):
            src = tag.get("src")
            srcs.append(src)
    data[title] = srcs

{'Hogwarts Legacy (Study Themes from the Original Video Game Soundtrack)': ['https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023/ynlgzwvomi/28.%20The%20Observation%20Deck.mp3',
  'https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023/sbggxvlify/02.%20Sorting%20Hat%20Ceremony.mp3',
  'https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023/wztmmqemmc/14.%20Hall%20of%20the%20Keepers.mp3',
  'https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023/qoewxbygal/30.%20Meeting%20with%20Professor%20Weasley.mp3',
  'https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-themes-from-the-original-video-game-soundtrack-2023/dojxvvyewa/41.%20Gringotts%20Vault.mp3',
  'https://eta.vgmtreasurechest.com/soundtracks/hogwarts-legacy-study-

In [22]:
with open('music.json', 'w') as f:
    json.dump(data, f, indent=4)